In [3]:
# Familiar imports
import numpy as np
import pandas as pd

# For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

# For training random forest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import mutual_info_regression
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, roc_auc_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from tqdm import tqdm
import optuna

from lightgbm import LGBMRegressor

In [4]:
# Load the training data
train = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv", index_col=0)
test = pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv", index_col=0)

# Preview the data
train.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,target
id,,,,,,,,,,,,,,,,,,,,,
0,0.106643,3.59437,132.8040,3.18428,0.081971,1.18859,3.73238,2.266270,2.09959,0.012330,...,1.09862,0.013331,-0.011715,0.052759,0.065400,4.211250,1.97877,0.085974,0.240496,0
1,0.125021,1.67336,76.5336,3.37825,0.099400,5.09366,1.27562,-0.471318,4.54594,0.037706,...,3.46017,0.017054,0.124863,0.154064,0.606848,-0.267928,2.57786,-0.020877,0.024719,0
2,0.036330,1.49747,233.5460,2.19435,0.026914,3.12694,5.05687,3.849460,1.80187,0.056995,...,4.88300,0.085222,0.032396,0.116092,-0.001688,-0.520069,2.14112,0.124464,0.148209,0
3,-0.014077,0.24600,779.9670,1.89064,0.006948,1.53112,2.69800,4.517330,4.50332,0.123494,...,3.47439,-0.017103,-0.008100,0.062013,0.041193,0.511657,1.96860,0.040017,0.044873,0
4,-0.003259,3.71542,156.1280,2.14772,0.018284,2.09859,4.15492,-0.038236,3.37145,0.034166,...,1.91059,-0.042943,0.105616,0.125072,0.037509,1.043790,1.07481,-0.012819,0.072798,1


In [5]:
y = train['target']
features = train.drop(['target'], axis=1)

X = features.copy()
X.shape

(600000, 100)

In [6]:
def create_pipeline(**kvargs):
    typ = kvargs.pop('typ')
    if typ == 'xgboost':
        model = XGBRegressor(**kvargs, n_jobs=3, tree_method='gpu_hist', gpu_id=0, eval_metric="auc")
    elif typ == 'lgbm':
        model = LGBMRegressor(**kvargs, n_jobs=3, device='gpu', metric = "auc")
    else:
        model = CatBoostRegressor(**kvargs)
    
    #pipeline = Pipeline(steps=[
    #    ('preprocessor', preprocessor),
    #    ('model', model)
    #])
    return model

In [7]:
def objective(trial, X, y, typ):
    #n_estimators = trial.suggest_int("n_estimators", 500, 5000)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.99)
    stopping_rounds = 400
    params={'typ':typ}
    if typ == 'xgboost':
        params['booster']='gbtree'
        params['n_estimators'] = trial.suggest_int("n_estimators", 5000, 20000)
        params['gamma'] = trial.suggest_float('gamma', 0, 100)
        params['max_depth'] = trial.suggest_int('max_depth', 1, 15)
        params['min_child_weight']=trial.suggest_float('min_child_weight', 0, 10)
        params['subsample']=trial.suggest_float('subsample', 0.1, 1)
        params['lambda']=trial.suggest_float('lambda', 1, 5)
        params['alpha']=trial.suggest_float('alpha', 0, 10)
    elif typ=='lgbm':
        params['n_estimators'] = trial.suggest_int("n_estimators", 500, 10000)
        params['max_depth']=trial.suggest_int('max_depth', 2, 10)
        params['num_leaves']=trial.suggest_int('num_leaves', 2, 100)
        params['reg_alpha']=trial.suggest_float('reg_alpha', 0, 10)
        params['reg_lambda']=trial.suggest_float('reg_lambda', 0, 10)
        params['min_data_in_leaf']=trial.suggest_int('min_data_in_leaf', 50, 1000)
    else:
        params['iterations'] = trial.suggest_int("iterations", 500, 5000)
        params['depth'] = trial.suggest_int("depth", 3, 15)
        params['l2_leaf_reg']=trial.suggest_float('l2_leaf_reg', 0.01, 100)
        params['bagging_temperature']=trial.suggest_float('bagging_temperature', 0, 10)
    
    #score = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    val_pred = np.zeros(len(y))

    mse = []
    spl = 10
    kf = KFold(n_splits=spl, shuffle=True)
    for trn_idx, val_idx in tqdm(kf.split(X,y)):
        x_train_idx = X.iloc[trn_idx]
        y_train_idx = y.iloc[trn_idx]
        x_valid_idx = X.iloc[val_idx]
        y_valid_idx = y.iloc[val_idx]

        model = create_pipeline(learning_rate=learning_rate, **params)
        verbose = -1
        if typ == 'xgboost':
            verbose = False
        if typ == 'catboost':
            verbose = 0
        model.fit(x_train_idx, y_train_idx, early_stopping_rounds = stopping_rounds, eval_set=[(x_valid_idx, y_valid_idx)], verbose = verbose)
        mse.append(roc_auc_score(y_valid_idx, model.predict(x_valid_idx)))
    
    accuracy = sum(mse)/spl
    return accuracy

In [8]:
study_name = 'tps-nov2-lbgm'  # Unique identifier of the study.
study = optuna.create_study(study_name=study_name, storage='sqlite:///'+study_name+'.db', direction="maximize", load_if_exists=True)

[I 2021-11-03 09:08:13,416] A new study created in RDB with name: tps-nov2-lbgm


In [9]:
print(study.best_trial)

ValueError: Record does not exist.

In [10]:
study.optimize(lambda trial: objective(trial, X, y, "lgbm"),  n_trials=100)
print(study.best_trial)

0it [00:00, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=398, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=398
Training until validation scores don't improve for 400 rounds
Early stopping, best iteration is:
[617]	valid_0's auc: 0.739001


1it [00:07,  7.19s/it]

[LightGBM] [Warning] min_data_in_leaf is set=398, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=398
Training until validation scores don't improve for 400 rounds
Early stopping, best iteration is:
[404]	valid_0's auc: 0.738676


2it [00:14,  7.07s/it]

...

[LightGBM] [Warning] min_data_in_leaf is set=744, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=744
Training until validation scores don't improve for 400 rounds
Early stopping, best iteration is:
[1737]	valid_0's auc: 0.749583


9it [02:39, 17.54s/it]

[LightGBM] [Warning] min_data_in_leaf is set=744, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=744
Training until validation scores don't improve for 400 rounds
Early stopping, best iteration is:
[1933]	valid_0's auc: 0.747119


10it [02:58, 17.86s/it]
[I 2021-11-03 13:51:12,737] Trial 99 finished with value: 0.7457552485747803 and parameters: {'learning_rate': 0.0851835659657465, 'n_estimators': 5822, 'max_depth': 7, 'num_leaves': 7, 'reg_alpha': 5.744820277162601, 'reg_lambda': 4.711912231697072, 'min_data_in_leaf': 744}. Best is trial 19 with value: 0.7475782744060069.


FrozenTrial(number=19, values=[0.7475782744060069], datetime_start=datetime.datetime(2021, 11, 3, 9, 57, 24, 810475), datetime_complete=datetime.datetime(2021, 11, 3, 10, 3, 34, 972252), params={'learning_rate': 0.088202251342637, 'max_depth': 9, 'min_data_in_leaf': 622, 'n_estimators': 6675, 'num_leaves': 2, 'reg_alpha': 3.2551114904483622, 'reg_lambda': 4.3256459403144465}, distributions={'learning_rate': UniformDistribution(high=0.99, low=0.001), 'max_depth': IntUniformDistribution(high=10, low=2, step=1), 'min_data_in_leaf': IntUniformDistribution(high=1000, low=50, step=1), 'n_estimators': IntUniformDistribution(high=10000, low=500, step=1), 'num_leaves': IntUniformDistribution(high=100, low=2, step=1), 'reg_alpha': UniformDistribution(high=10.0, low=0.0), 'reg_lambda': UniformDistribution(high=10.0, low=0.0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=20, state=TrialState.COMPLETE, value=None)


In [11]:
from datetime import datetime
print(datetime.now())

2021-11-03 13:51:12.751799
